This is a replication of the original paper's experiments. It calculates the alpha values as described in a recent update to the original author's repo.

## Python version and libraries

In [6]:
!python3 --version

Python 3.8.13


## Tool Help

In [7]:
!python3 cs_classification.py -h > outputs/help.txt

## Contrast Subgraph Problem 1 (CSP1)

### Solver=SDP, charikar_projection, tsourakakis local search, a single contrast subgraph

In [8]:
!python3 cs_classification.py datasets/children/td/ datasets/children/asd/ --percentile 70 --percentile2 80 -k 5 -pre CSP1-SDP-Percentile-Children > outputs/CSP1-SDP-Percentile-Children.txt

In [9]:
!python3 cs_classification.py datasets/adolescents/td/ datasets/adolescents/asd/ --percentile 95 --percentile2 70  -k 5 -pre CSP1-SDP-Percentile-Adolescents > outputs/CSP1-SDP-Percentile-Adolescents.txt

In [10]:
!python3 cs_classification.py datasets/eyesclosed/td/ datasets/eyesclosed/asd/ --percentile 75 --percentile2 75 -k 5 -pre CSP1-SDP-Percentile-Eyesclosed > outputs/CSP1-SDP-Percentile-Eyesclosed.txt

In [11]:
!python3 cs_classification.py datasets/male/td/ datasets/male/asd/ --percentile 75 --percentile2 70 -k 5 -pre CSP1-SDP-Percentile-Male > outputs/CSP1-SDP-Percentile-Male.txt

## Contrast Subgraph Problem 2 (CSP2)

### Solver=SDP, charikar_projection, tsourakakis local search, a single contrast subgraph

In [12]:
!python3 cs_classification.py datasets/children/td/ datasets/children/asd/ -p 2 --percentile 70 -k 5 -pre CSP2-SDP-Percentile-Children > outputs/CSP2-SDP-Percentile-Children.txt

In [13]:
!python3 cs_classification.py datasets/adolescents/td/ datasets/adolescents/asd/ -p 2 --percentile 75 -k 5 -pre CSP2-SDP-Percentile-Adolescents > outputs/CSP2-SDP-Percentile-Adolescents.txt

In [14]:
!python3 cs_classification.py datasets/eyesclosed/td/ datasets/eyesclosed/asd/ -p 2 --percentile 70 -k 5 -pre CSP2-SDP-Percentile-Eyesclosed > outputs/CSP2-SDP-Percentile-Eyesclosed.txt

In [15]:
!python3 cs_classification.py datasets/male/td/ datasets/male/asd/ -p 2 --percentile 70 -k 5 -pre CSP2-SDP-Percentile-Male > outputs/CSP2-SDP-Percentile-Male.txt

/home/kenns/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
